In [ ]:
# # Install necessary libraries
# !pip install transformers datasets

# # Import libraries
# import random
# from datasets import load_dataset
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# # Step 1: Load the MMLU Dataset
# # dataset = load_dataset("hendrycks_test", "abstract_algebra", split="validation")  # Example subset: "abstract_algebra"
# dataset = load_dataset("hails/mmlu_no_train", "all")

# # Step 2: Preprocess the Data
# def preprocess_mmlu(data):
#     """
#     Prepares the MMLU dataset by randomly masking one incorrect option.
#     """
#     processed_data = []
#     for example in data['test']: # Access the 'test' split of the dataset
#         question = example["question"]
#         answer = example["answer"]  # The correct answer (A, B, C, or D)
#         # Assuming 'choices' contains the options as a list
#         options = example["choices"]

#         # Map the correct answer letter to its index
#         correct_index = example["answer"]

#         # Mask one incorrect option randomly
#         incorrect_indices = [i for i in range(len(options)) if i != correct_index]
#         masked_index = random.choice(incorrect_indices)
#         masked_options = options.copy()
#         masked_options[masked_index] = "______"

#         processed_data.append({
#             "question": question,
#             "options": options,
#             "masked_options": masked_options,
#             "correct_index": correct_index,
#             "masked_index": masked_index,
#         })
#     return processed_data

# processed_data = preprocess_mmlu(dataset)

# # Step 3: Load GPT-2 Model and Tokenizer
# model_name = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Step 4: Predict the Masked Option
# def predict_with_gpt2(question, masked_options, tokenizer, model):
#     """
#     Uses GPT-2 to predict the masked option in a multiple-choice question.
#     """
#     # Prepare the input prompt
#     input_prompt = (
#         f"Question: {question}\n"
#         f"Options: {' | '.join(masked_options)}\n"
#         f"The missing option is:"
#     )

#     # Tokenize input
#     input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids

#     # Generate prediction
#     with torch.no_grad():
#         output_ids = model.generate(input_ids, max_length=input_ids.shape[1] + 10, num_return_sequences=1)
#     output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

#     # Extract the predicted option
#     predicted_text = output_text.replace(input_prompt, "").strip()
#     return predicted_text

# # Step 5: Evaluate the Model on MMLU
# def evaluate_gpt2(data, tokenizer, model):
#     """
#     Evaluates GPT-2 on the MMLU dataset by calculating accuracy on masked options.
#     """
#     correct_predictions = 0
#     total_predictions = 0

#     for example in data:
#         question = example["question"]
#         masked_options = example["masked_options"]
#         correct_option = example["options"][example["correct_index"]]
#         masked_option = example["options"][example["masked_index"]]
#         print(f"the example is: {example}")

#         # Predict the masked option
#         predicted_option = predict_with_gpt2(question, masked_options, tokenizer, model)

#         print(f"predicted option is: {predicted_option}")
#         # Check if the prediction matches the masked option
#         if predicted_option.strip().lower() == masked_option.strip().lower():
#             correct_predictions += 1
#         total_predictions += 1

#     # Calculate accuracy
#     accuracy = (correct_predictions / total_predictions) * 100
#     return accuracy

# # Step 6: Run Evaluation
# accuracy = evaluate_gpt2(processed_data[:50], tokenizer, model)  # Test on the first 50 examples for faster evaluation
# print(f"GPT-2 Accuracy on MMLU: {accuracy:.2f}%")

## Using cosing similarity
# Install necessary libraries
!pip install transformers datasets sentence-transformers

# Import libraries
import random
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util

# Step 1: Load the MMLU Dataset
dataset = load_dataset("hails/mmlu_no_train", "all")

# Step 2: Preprocess the Data
def preprocess_mmlu(data):
    """
    Prepares the MMLU dataset by randomly masking one incorrect option.
    """
    processed_data = []
    for example in data['test']:  # Access the 'test' split of the dataset
        question = example["question"]
        options = example["choices"]
        correct_index = example["answer"]

        # Mask one incorrect option randomly
        incorrect_indices = [i for i in range(len(options)) if i != correct_index]
        masked_index = random.choice(incorrect_indices)
        masked_options = options.copy()
        masked_options[masked_index] = "______"

        processed_data.append({
            "question": question,
            "options": options,
            "masked_options": masked_options,
            "correct_index": correct_index,
            "masked_index": masked_index,
        })
    return processed_data

processed_data = preprocess_mmlu(dataset)

# Step 3: Load GPT-2 Model and Tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 4: Load SentenceTransformer for Semantic Matching
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Step 5: Predict the Masked Option
def predict_with_gpt2(question, masked_options, tokenizer, model):
    """
    Uses GPT-2 to predict the masked option in a multiple-choice question.
    """
    # Prepare the input prompt
    input_prompt = (
        f"Question: {question}\n"
        f"Options: {' | '.join(masked_options)}\n"
        f"The missing option is:"
    )

    # Tokenize input
    input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids

    # Generate prediction
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=input_ids.shape[1] + 10, num_return_sequences=1)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract the predicted option
    predicted_text = output_text.replace(input_prompt, "").strip()
    return predicted_text

# Step 6: Semantic Matching
def semantic_match(predicted_output, masked_option, embedder, threshold=0.8):
    """
    Determines if the predicted output semantically matches the masked option using embeddings.
    """
    # Compute embeddings
    predicted_embedding = embedder.encode(predicted_output, convert_to_tensor=True)
    masked_embedding = embedder.encode(masked_option, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.cos_sim(predicted_embedding, masked_embedding).item()
    print(f"Similarity: {similarity:.2f}")  # Debugging

    # Check if similarity exceeds the threshold
    return similarity >= threshold

# Step 7: Evaluate the Model on MMLU with Semantic Matching
def evaluate_gpt2_with_semantics(data, tokenizer, model, embedder, threshold=0.8):
    """
    Evaluates GPT-2 on the MMLU dataset using semantic matching for masked options.
    """
    correct_predictions = 0
    total_predictions = 0

    for example in data:
        question = example["question"]
        masked_options = example["masked_options"]
        masked_option = example["options"][example["masked_index"]]

        # Predict the masked option
        predicted_output = predict_with_gpt2(question, masked_options, tokenizer, model)
        print(f"Predicted output: {predicted_output}")
        print(f"Masked option: {masked_option}")

        # Check for a semantic match
        if semantic_match(predicted_output, masked_option, embedder, threshold):
            correct_predictions += 1
        total_predictions += 1

    # Calculate accuracy
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

# Step 8: Run Evaluation
accuracy_with_semantics = evaluate_gpt2_with_semantics(processed_data[:50], tokenizer, model, embedder, threshold=0.8)
print(f"GPT-2 Accuracy on MMLU with Semantic Matching: {accuracy_with_semantics:.2f}%")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Predicted output: The president would veto the bill.
The missing
Masked option: The majority of American citizens support the proposed legislation.
Similarity: 0.26


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted output: The missing option is:
The missing option
Masked option: Labor unions
Similarity: 0.02


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted output: The Constitution does not provide for the transfer of
Masked option: veto legislative bills of attainder
Similarity: 0.37
Predicted output: The bill would have to be passed by the
Masked option: An adjournment
Similarity: 0.28
GPT-2 Accuracy on MMLU with Semantic Matching: 0.00%
